In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [13]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(name="ai-professor", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'id': 1,
  'professor_name': 'Dr. Sarah Johnson',
  'department': 'Physics',
  'course_code': 'PHY301',
  'rating': 4.5,
  'difficulty': 3.8,
  'comment': 'Challenging but fair. Great at explaining complex concepts.',
  'would_take_again': True,
  'date': '2024-03-15',
  'grade': 'A-'},
 {'id': 2,
  'professor_name': 'Prof. Michael Lee',
  'department': 'Computer Science',
  'course_code': 'CS201',
  'rating': 3.2,
  'difficulty': 4.5,
  'comment': "Tough grader, but you'll learn a lot if you put in the effort.",
  'would_take_again': False,
  'date': '2024-02-28',
  'grade': 'B'},
 {'id': 3,
  'professor_name': 'Dr. Emily Chen',
  'department': 'Biology',
  'course_code': 'BIO102',
  'rating': 4.8,
  'difficulty': 2.5,
  'comment': 'Engaging lectures and very approachable. Highly recommended!',
  'would_take_again': True,
  'date': '2024-03-10',
  'grade': 'A+'},
 {'id': 4,
  'professor_name': 'Prof. David Brown',
  'department': 'History',
  'course_code': 'HIS205',
  'rating': 3.9

In [9]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['comment'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": str(review['id']),
        "metadata": {
            "professor_name": review['professor_name'],
            "department": review['department'],
            "course_code": review['course_code'],
            "rating": review['rating'],
            "difficulty": review['difficulty'],
            "comment": review['comment'],
            "would_take_again": review['would_take_again'],
            "date": review['date'],
            "grade": review["grade"]
        }
    })

In [6]:
processed_data[0]

{'values': [-0.036637086,
  -0.0059646214,
  -0.009802028,
  0.019726485,
  0.039269324,
  -0.005788629,
  -0.042727962,
  0.035198536,
  0.017706394,
  0.05013496,
  0.07082558,
  -0.014201844,
  -0.016604528,
  -0.037157413,
  0.012373051,
  -0.0108273765,
  -0.026628459,
  -0.0025098086,
  0.039759044,
  0.03945297,
  0.035474002,
  0.0076939412,
  0.025266428,
  0.07253959,
  -0.027623199,
  -0.041626096,
  0.00055093365,
  -0.015058853,
  0.018609313,
  0.02910766,
  0.068499416,
  -0.017078942,
  0.0003278342,
  -0.05273659,
  7.1317685e-05,
  0.07553912,
  -0.018792957,
  0.042238243,
  0.04300343,
  0.010131057,
  0.028143525,
  0.025281731,
  -0.030393172,
  -0.024730798,
  0.03599433,
  -0.015158326,
  -0.016053593,
  -0.0019301806,
  0.016374972,
  0.0052147396,
  -0.027868059,
  0.04713543,
  0.043646187,
  0.0022400809,
  -0.042085208,
  0.036269795,
  0.02613874,
  0.03599433,
  -0.030959409,
  0.0012281229,
  0.03146443,
  -0.03290298,
  0.0072883926,
  -0.010865635,
  -

In [14]:
index = pc.Index("ai-professor")
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

{'upserted_count': 20}

In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}